# Building a hierarchical multi-agent travel assistant with CrewAI

This notebook demonstrates how to create a travel assistant using multi-agent collaboration with the CrewAI framework. Specifically, you will use this assistant to generate articles on the top activities and attractions for a specific location. 

## Structure of the travel assistant
This notebook provides step-by-step guidance for creating a travel assistant consisting of the following agents:

1. Researcher Agent - is responsible for gathering information on the top 5 attractions and activities in a specific location
2. Content Writer Agent - is responsible for writing an engaging article based on the information of the top 5 attractions
3. Editor Agent - is responsible for improving the flow and language use within the article

## Setup

Start by installing some of the required packages, including CrewAI for building multi-agent solutions, Langchain for pre-built tool components, and Tavily for its web search API.

In [ ]:
%pip install -r crewai-requirements.txt -qU

<div class="alert alert-block alert-info">
<b>Important:</b> restart the kernel before proceeding with the next cells.
</div>

Now you can import all of the packages you will need for this lab. This includes the packages described above, but also textwrap for keeping the prompts readable within the notebook, and pydantic for validating input.

In [ ]:
from crewai import LLM, Agent, Crew, Task, Process
from textwrap import dedent
from crewai_tools import SerperDevTool

Follow the instructions in [this lab](https://catalog.us-east-1.prod.workshops.aws/workshops/eb7af74b-184e-47d2-9277-4f59b4c91df5/en-US/2-llm-deployment-with-aws/2-sagemaker) to create a SageMaker real-time endpoint hosting the [DeepSeek R1 Distill-Llama 8B](https://huggingface.co/deepseek-ai/DeepSeek-R1) model. Although you will use this specific model for this lab, CrewAI also supports other models hosted through [Amazon SageMaker](https://docs.crewai.com/concepts/llms#amazon-sagemaker) or [Amazon Bedrock](https://docs.crewai.com/concepts/llms#aws-bedrock). While the SageMaker endpoint is being created, you can copy the name of the endpoint and replace the text `INSERT ENDPOINT NAME` in the code below with the endpoint name.

When setting up an LLM model to use with CrewAI, you can also set model parameters. In this case, we set `temperature` to 0.7 to balance creativity with accuracy. We also set the `max_tokens` to 4096 to allow for longer responses.

For the LLM attached to agents with tools, we will need a model capable of supporting function calling. You can use [Amazon Nova Pro](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-nova.html), but another valid open-source option could be Meta Llama 3.1 70B Instruct (`bedrock/invoke/meta.llama3-1-70b-instruct-v1:0`). 

Both Bedrock models will require to be called using [Cross-Region Inference](https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html).

In [ ]:
from crewai import LLM

# We will use the reasoning LLM as the "manager" LLM - the supervisor
reasoning_llm = LLM(
    model="sagemaker/INSERT ENDPOINT NAME",
    temperature=0.7, max_tokens=2*1024,
)

# We will leverage a model good for function calling as task executor instead
function_calling_llm = LLM(
    model="bedrock/us.amazon.nova-pro-v1:0",
    temperature=0, max_tokens=5*1024
)

## Define the Tools

In [ ]:
from getpass import getpass
import os
from dotenv import load_dotenv

# Make sure the Serper API Key is configured
load_dotenv()
if not os.getenv("SERPER_API_KEY"):
    os.environ["SERPER_API_KEY"] = getpass("Please enter your Serper API key: ")

search_tool = SerperDevTool(
    search_url="https://google.serper.dev/search",
    n_results=6,
)

<div class="alert alert-block alert-warning">
<h3><b>Having trouble with Serper API Key?</b></h3>
Run the next few cells to create a custom Tool to use DuckDuckGo to perform searches. Less stable than Serper.<b>If you've managed to get a Serper API Key</b>, ignore the next two cells.
</div>

In [ ]:
%pip install duckduckgo-search -qU

In [ ]:
from crewai.tools import tool
from duckduckgo_search import DDGS

@tool('DuckDuckGoSearch')
def search_tool(search_query: str):
    """Search the web for information on a given topic"""
    return DDGS().text(search_query, max_results=5)

## Create the crew

A crew of one agent is not very interesting, so let's add a second agent to the crew. Using the same techniques as above, create a second agent which specializes in writing informative top 5 travel listicles (a type of article which contains a list). 

In [ ]:
# Define the manager agent
supervisor_agent = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
    verbose=True,
    tools=[],
    llm=reasoning_llm,
    max_iter=4
)

In [ ]:
researcher_agent = Agent(
    role="Travel Researcher",
    goal="Research and compile interesting activities and attractions for a given location",
    backstory=dedent(
        """You are an experienced travel researcher with a knack for 
        discovering both popular attractions and hidden gems in any 
        location. Your expertise lies in gathering comprehensive 
        information about various activities, their historical 
        significance, and practical details for visitors.
        """
    ),
    allow_delegation=False,
    verbose=True,
    llm=function_calling_llm,
    max_iter=4,
    tools=[search_tool],
)

research_task = Task(
    description=dedent(
        """Research and compile a list of at least 5 interesting 
        activities and attractions in {location}. Include a mix of 
        popular tourist spots and lesser-known local favorites. For 
        each item, provide:
        1. Name of the attraction/activity
        2. Brief description (2-3 sentences)
        3. Why it's worth visiting
        Your final answer should be a structured list of these items.
        """
    ),
    agent=researcher_agent,
    expected_output="Structured list of 5+ attractions/activities",
)

In [ ]:
content_writer_agent = Agent(
    role="Travel Content Writer",
    goal="Create engaging and informative content for the top 5 listicle",
    backstory=dedent(
        """You are a skilled travel writer with a flair for creating 
        captivating content. Your writing style is engaging, 
        informative, and tailored to inspire readers to explore new 
        destinations. You excel at crafting concise yet compelling 
        descriptions of attractions and activities."""
    ),
    allow_delegation=False,
    verbose=True,
    llm=function_calling_llm,
    max_iter=4,
    tools=[],
)

write_task = Task(
    description=dedent(
        """Create an engaging top 5 listicle article about things to 
        do in {location}. Use the research provided to:
        1. Write a catchy title and introduction (100-150 words)
        2. Select and write about the top 5 activities/attractions
        3. For each item, write 2-3 paragraphs (100-150 words total)
        4. Include a brief conclusion (50-75 words)

        Ensure the content is engaging, informative, and inspiring. 
        Your final answer should be the complete listicle article.
        """
    ),
    agent=content_writer_agent,
    expected_output="Complete top 5 listicle article",
)

In [ ]:
editor_agent = Agent(
    role="Content Editor",
    goal="Ensure the listicle is well-structured, engaging, and error-free",
    backstory=dedent(
        """You are a meticulous editor with years of experience in 
        travel content. Your keen eye for detail helps polish articles 
        to perfection. You focus on improving flow, maintaining 
        consistency, and enhancing the overall readability of the 
        content while ensuring it appeals to the target audience."""
    ),
    allow_delegation=False,
    verbose=True,
    llm=function_calling_llm,
    max_iter=4,
    tools=[],
)

edit_task = Task(
    description=dedent(
        """Review and edit the top 5 listicle article about things to 
        do in {location}. Focus on:
        1. Improving the overall structure and flow
        2. Enhancing the engagement factor of the content
        3. Ensuring consistency in tone and style
        4. Correcting any grammatical or spelling errors

        Do not change the content itself. Only edit it for higher quality.
        Your final answer should be the polished, publication-ready 
        version of the article.
        """
    ),
    agent=editor_agent,
    expected_output="Edited and polished listicle article about {location}",
)

After creating your second and third agent, you will need to add them both to your crew. Tasks can be executed sequentially (i.e. always in the order which they are defined) or hierarchically (i.e. tasks are assigned based on agent roles). In this case, the process for writing a travel article is always the same, so we will use sequential.

In [ ]:
crew = Crew(
    agents=[researcher_agent, content_writer_agent, editor_agent],
    tasks=[research_task, write_task, edit_task],
    manager_agent=supervisor_agent,
    # manager_llm=reasoning_llm,
    process=Process.hierarchical,
    verbose=True,
)

Next, test your crew!

In [ ]:
inputs = {
    'location': 'Hoenderloo, Netherlands'
}
listicle_result = crew.kickoff(inputs=inputs)
print(listicle_result)